In [ ]:
import numpy as np

In [ ]:
class Evolutionary():
    def __init__(self,pop, mprob, rprob, geneSize, fitnessFunction):
        self.pop = np.zeros(pop)
        self.mutatProb = mprob
        self.recomProb = rprob
        self.geneSize
        self.fitnessFunction = fitnessFunction
    
    def populate(self):
        for i in range(len(self.pop)):
            prob = [5/len(self.pop), len(self.pop)-10/len(self.pop),5/len(self.pop)]
            newTeam = np.random.choice([-1,0,1], self.geneSize, p=prob)
            newRole = np.random.choice([0,1,2,3,4], self.geneSize)
            new = np.zeros(self.geneSize)
            for j in range(self.geneSize):
                new[j] = [newTeam[j], newRole[j]]
                
            self.pop[i] = new
        fixUsing():
        return
    
    def fixUsing():
        for i in range(len(self.pop)):
            count = countUsing(self.pop[i])
            if count < 10:
                for j in range(self.geneSize):
                    if count >= 10:
                        break
                    else:
                        if self.pop[i][j] == 0:
                            self.pop[i][j] == np.random.choice([-1,1])
    def countUsing(self, i):
        count = 0
        for j in i:
            if j[0] == -1 | j[0] == 1:
                count += 1
        return count
            
    def step(self):
        a = np.random.randint(len(self.pop))
        b = np.random.randint(len(self.pop))
        while(a != b):
            b = np.random.randint(len(self.pop))
        if self.fitnessFunction(self.pop[a]) > self.fitnessFunction(self.pop[b]):
            winner = a
            loser = b
        else:
            winner = b
            loser = a
        
        for i in range(self.geneSize):
            if(np.random.random_sample() > self.recomProb):
                self.pop[loser][i] = self.pop[winner][i]
        for i in range(self.geneSize):
            if(np.random.random_sample() > self.mutatProb):
                
                prob = [5/len(self.pop), len(self.pop)-10/len(self.pop),5/len(self.pop)]
                newTeam = np.random.choice([-1,0,1], p=prob)
                newRole = np.random.choice([0,1,2,3,4])
                new = [newTeam, newRole]
                if new != self.pop[loser][i]:
                    self.pop[loser][i] = new
        fixUsing()

1